In [1]:
#cnet_model.ipynb 
# 
#by Joe Hahn
#joe.hahn@oracle.com
#30 July 2018
#
#train a cnet layer on the XO dataset

In [2]:
#set system parameters
x_half_width = 0.5
radius = 3.5
box_half_width = 7.1
jitter = 1.0
rn_seed = 15
initial_id = 0

In [3]:
#start time
import time as tm
time_start = tm.time()

In [4]:
#generate balanced training data
N_dots = 26700
import numpy as np
rn_state = np.random.RandomState(seed=rn_seed)
from helper_fns import *
df = make_xo_df(N_dots, initial_id, x_half_width, radius, box_half_width, jitter)
df = rebalance_df(df)
print df.groupby('class')['id'].count()
train = df
train.tail(5)

class
B    3344
O    3344
X    3344
Name: id, dtype: int64


,id,x,y,r,angle,class,X_score,O_score,B_score
10027,3953,-6.050939,-2.436575,6.523094,-2.758780,B,0.0,0.0,1.0
10028,863,1.994081,2.602057,3.278271,0.916914,O,0.0,1.0,0.0
10029,8564,1.660073,-0.244175,1.677934,-0.146040,O,0.0,1.0,0.0
10030,19766,0.055299,1.422387,1.423462,1.531938,X,1.0,0.0,0.0
10031,4825,3.603196,-4.416232,5.699661,-0.886436,X,1.0,0.0,0.0


In [5]:
#select x-features and y = target variables = class-scores
x_cols = ['x', 'y']
y_cols = ['O_score', 'X_score', 'B_score']
x = train[x_cols].values
y = train[y_cols].values
print 'x.shape = ', x.shape
print 'y.shape = ', y.shape
x

x.shape =  (10032, 2)
y.shape =  (10032, 3)


array([[ 0.00875399, -1.36284745],
       [ 0.76575749,  1.32003586],
       [-2.59736693, -2.92864857],
       ..., 
       [ 1.66007301, -0.24417544],
       [ 0.05529941,  1.42238712],
       [ 3.6031964 , -4.41623226]])

In [6]:
#prep plots
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
sns.set(font_scale=1.5, font='DejaVu Sans')

In [7]:
#done
time_stop = tm.time()
print 'execution time (minutes) = ', (time_stop - time_start)/60.0

execution time (minutes) =  0.024538731575
